#Dataset
it consists of three folders test,train and single prediction

Dataset
The problem we will consider here is classifying 4 different building defects . The dataset we will use is collection of 866 images of 4 types of defects 

The labels in this dataset are the following:
<br>
 **cracks,flakes,roof,spall**  



In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import cv2
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import os
os.getcwd()

'C:\\Users\\haris\\Desktop\\cnn'

In [3]:
os.chdir('c:\\Users\\haris\\Desktop\\cnn')

###Data Augmentation
To avoid overfitting and create a larger dataset from a smaller one we can use a technique called data augmentation. This is simply performing random transofrmations on our images so that our model can generalize better. These transformations can be things like compressions, rotations, stretches and even color changes. 

Fortunately, keras can help us do this.



In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
x_train = train_datagen.flow_from_directory('Dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                     class_mode = 'categorical')
x_test = test_datagen.flow_from_directory('Dataset/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 662 images belonging to 4 classes.
Found 202 images belonging to 4 classes.


In [6]:
print(x_train.class_indices)

{'crack': 0, 'flakes': 1, 'roof': 2, 'spalling': 3}


##CNN Architecture
A common architecture for a CNN is a stack of Conv2D and MaxPooling2D layers followed by a few denesly connected layers. To idea is that the stack of convolutional and maxPooling layers extract the features from the image. Then these features are flattened and fed to densly connected layers that determine the class of an image based on the presence of features.

We will start by building the **Convolutional Base and then add layers**.

Layer 1

The input shape of our data will be 32, 32, 3 and we will process 32 filters of size 3x3 over our input data. We will also apply the activation function relu to the output of each convolution operation.

Layer 2

This layer will perform the max pooling operation using 2x2 samples and a stride of 2.

Other Layers

i didnt keep extra layers for depth of CNN network to make it simple and not too complex

In [7]:
model=Sequential()

In [8]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation="relu"))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
model.add(MaxPooling2D(pool_size = (2, 2)))

In [10]:
model.add(Flatten())

In [11]:
#Add Hidden Layer
model.add(Dense(units=128,init="random_uniform",activation="relu"))

c:\users\haris\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, activation="relu", kernel_initializer="random_uniform")`
  


In [12]:
#Add Output layer
model.add(Dense( units=4,init="random_uniform",activation="softmax"))

c:\users\haris\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=4, activation="softmax", kernel_initializer="random_uniform")`
  


In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total params: 3,937,796
Trainable params: 3,937,796
Non-trainable params: 0
_________________________________________________________________


##Training
Now we will train and compile the model using the recommended hyper paramaters from tensorflow.

*Note: This will take much longer based on number of epochs!*

In [14]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [15]:
model.fit_generator(x_train,
                         steps_per_epoch = 190,
                         epochs = 15,
                         validation_data = x_test,
                         validation_steps = 63)


Epoch 1/15
190/190 [==============================] - 53s 281ms/step - loss: 1.1704 - accuracy: 0.4912 - val_loss: 1.2051 - val_accuracy: 0.3317
Epoch 2/15
190/190 [==============================] - 54s 287ms/step - loss: 0.8917 - accuracy: 0.6397 - val_loss: 1.1116 - val_accuracy: 0.3911
Epoch 3/15
190/190 [==============================] - 44s 229ms/step - loss: 0.6916 - accuracy: 0.7322 - val_loss: 2.0580 - val_accuracy: 0.4059
Epoch 4/15
190/190 [==============================] - 42s 221ms/step - loss: 0.5242 - accuracy: 0.8048 - val_loss: 1.1136 - val_accuracy: 0.4752
Epoch 5/15
190/190 [==============================] - 44s 231ms/step - loss: 0.4065 - accuracy: 0.8587 - val_loss: 2.0943 - val_accuracy: 0.4307
Epoch 6/15
190/190 [==============================] - 43s 226ms/step - loss: 0.3162 - accuracy: 0.8863 - val_loss: 2.1421 - val_accuracy: 0.4950
Epoch 7/15
190/190 [==============================] - 49s 258ms/step - loss: 0.2248 - accuracy: 0.9244 - val_loss: 3.2910 - val_a

#Saving Model

In [16]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [17]:
model.save("model.h5")

#Loading Model

In [18]:
from keras.models import model_from_json 

# opening and store file in a variable

json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model

loaded_model = model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights("model.h5")
print("Loaded Model from disk")

# compile and evaluate loaded model

loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#graph = tf.get_default_graph()
graph = tf.compat.v1.get_default_graph()

print(graph)

Loaded Model from disk


#Single Prediction

In [19]:
from keras.models import load_model
import numpy as np
import cv2

In [20]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('Dataset/single_prediction/4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict_classes(test_image)
x_train.class_indices
print(result)
result_array = model.predict(test_image)
print(result_array)

if result== 0:
  prediction = 'Cracks'
elif result == 1:
  prediction = 'Flakes'
elif result == 2:
  prediction = 'roof'
else:
  prediction = 'Spalling'


[1]
[[0. 1. 0. 0.]]


In [21]:
print(prediction)

Flakes


##Evaluating the Model
We can determine how well the model performed by looking at it's performance on the test data set.

In [22]:
final_loss, final_accuracy = model.evaluate(x_test, steps = 63)
print('Final loss: {:.2f}'.format(final_loss))
print('Final accuracy: {:.2f}%'.format(final_accuracy * 100))

63/63 [==============================] - 9s 135ms/step
Final loss: 5.24
Final accuracy: 49.50%


In [23]:
# import matplotlib.pyplot as plt
# IMG_INDEX = 7  # change this to look at other images

# plt.imshow(x_train[IMG_INDEX] ,cmap=plt.cm.binary)
# plt.xlabel(class_names[train_labels[IMG_INDEX][0]])
# plt.show()

In [24]:
#frame=cv2.imread('C:\\Users\\haris\\Desktop\\Section 40 - Convolutional Neural Networks (CNN)\\Dataset\\test_set\\cracks\\c5.jpg')
#data = detect(frame)


In [25]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.ylabel('Loss (training and validation)')
# plt.xlabel('Training Steps')
# plt.ylim([0,50])
# plt.plot(hist['loss'])
# plt.plot(hist['val_loss'])
# plt.figure()
# plt.ylabel('Accuracy (training and validation)')
# plt.xlabel('Training Steps')
# plt.ylim([0,1])
# plt.plot(hist['acc'])
# plt.plot(hist['val_acc'])